# Class definition

In [1]:
class LogisticRegression:
    """
    LogisticRegression fits a regression model with k regressors, to minimize the cross entropy/
    log loss error between the observed target variable in the dataset, and the target predicted 
    by the approximation.
    
    Needed packages:
    import numpy as np

    Parameters
    ----------
    fit_intercept : bool, default = True,
        Specifies if a constant should be added to the regression.
    
    batch_size : int, default = 1000
        Size of the batch in SGD loop.
    
    no_epochs : int, default = 10000
        Number of epochs taken for the SGD to converge.
        
    learning_rate : float, default = 0.01
        Learning rate in SGD.
    
    tolerance : float, default = 1e-10
        Tolerance for stopping criteria.
    
    verbose : bool, default = False
        Boolean to print or suppress epochs in SGD loop.
        
        
    Attributes
    ----------
    n_features : int
        Number of features seen during method `fit`, excluding intercept.
        
    classes_ : ndarray of shape n_classes
        A list of class labels seen during method `fit`.
    
    n_classes : int
        Number of target classes seen during method `fit`.
    
    coefficients : ndarray of shape n_features + 1
        Fitted coefficient of the features in the decision function. When fit_intercept is
        set to true, coefficients includes bias term, hence first value is intercept.

    intercept : ndarray of shape 1
        Fitted intercept, if added to the decision function. First value in coefficients.
        Set to 0.0 if fit_intercept = False.
        
    residuals : array of shape N
        Estimated residuals, defined as the difference between the predicted,
        and the true y-value.
        
    deviance_residuals : array of shape N
        Estimated deviance residuals.

    
    Methods
    ----------
    fit(X,y):
        Fit Logistic regression model.
        
    predict(X, boundary=0.5):
        Predict using the fitted parameters of this Logistic Regression estimator.
        
    predict_prob(X, boundary=0.5):
        Predict probability using the fitted parameters of this Logistic Regression estimator.
        
    accuracy(y, y_pred)
        Print accuracy of fitted Logistic Regression estimator
    
        
    Examples
    --------
    >>> import numpy as np
    >>> X = np.array([[-1, 1], [1, 2], [3, 4], [-3, 1], [-1, 0]])
    >>> y = np.array([1, 0, 1, 0, 0])

    >>> regfit = LogisticRegression(verbose=False, no_epochs=100000, tolerance=1e-10, fit_intercept=True).fit(X,y)
    >>> regfit.n_features
    2
    >>> regfit.classes
    array([0, 1])
    >>> regfit.n_classes
    2
    >>> regfit.coefficients
    array([[-1.93422523],
           [ 0.04175192],
           [ 0.93863707]])
    >>> regfit.intercept
    array([-1.93422523])
    >>> regfit.residuals
    array([[-0.73833641],
           [ 0.49620032],
           [-0.12503628],
           [ 0.24585473],
           [ 0.1217484 ]])
    >>> regfit.deviance_residuals
    array([[-1.63749542],
           [ 1.17096248],
           [-0.5168614 ],
           [ 0.75122601],
           [ 0.50955307]])
    >>> regfit.predict(X)
    array([0, 0, 1, 0, 0])
    >>> regfit.predict_prob(X)
    array([[0.73833645, 0.26166355],
           [0.50379972, 0.49620028],
           [0.12503625, 0.87496375],
           [0.75414522, 0.24585478],
           [0.87825167, 0.12174833]])
    >>> regfit.accuracy(y, regfit.y_pred)
    0.8
    """
    
    def __init__(
        self,
        fit_intercept = True,
        batch_size = 1000, 
        no_epochs = 10000, 
        learning_rate = 0.01,
        tolerance = 1e-10,
        verbose = False,
    ):
        self.fit_intercept = fit_intercept
        self.batch_size = batch_size
        self.no_epochs = no_epochs
        self.learning_rate = learning_rate
        self.tolerance = tolerance
        self.verbose = verbose
        
    @staticmethod
    def sigmoid(z):
            return 1/(1 + np.exp(-z))
        
    @staticmethod    
    def log_loss(y, y_hat):
            return -np.mean(y*(np.log(y_hat)) + (1-y)*np.log(1-y_hat))
        
    @staticmethod        
    def gradients(X, y, y_hat):
            return (1/len(y))* X.T@(y_hat - y)
        
    def fit(self, X, y):
        """
        Fit Logistic regression model.

        Parameters
        ----------
        X : array of shape Nxk (Training data)
        y : array of shape N (Target values)

        Returns
        -------
        self : object
            Fitted Estimator.
        """
        # Define number of features and classes
        self.n_features = len(X[0])
        self.classes = np.unique(y)
        self.n_classes = len(self.classes)
        
        if self.fit_intercept == True:
            X = np.append(np.ones((len(X),1)), X, axis=1)

        # N, k: sample size, number of features
        N, k = X.shape

        # Initialize weights
        self.coefficients = np.zeros((k,1))

        # Reshape y
        y = y.reshape(N,1)

        # Empty list to store losses
        self.log_losses = []
        
        # Training loop
        for epoch in range(self.no_epochs):
            if self.verbose == True:
                print(f'Running epoch: {epoch}/{self.no_epochs}')
            for i in range((N-1)//self.batch_size + 1):

                # Define batches, SGD
                start_i = i*self.batch_size
                end_i = start_i + self.batch_size
                xb = X[start_i:end_i]
                yb = y[start_i:end_i]

                # Calculate hypothesis/prediction.
                y_hat = self.sigmoid(xb@self.coefficients)

                # Get the gradients of loss w.r.t parameters.
                grad = self.gradients(xb, yb, y_hat)

                # Update the parameters.
                self.coefficients -= self.learning_rate*grad

            # Find previous log_loss
            try:
                l_prev = l.copy()
            except:
                l_prev = np.nan

            # Calculate log_loss
            l = self.log_loss(y, self.sigmoid(X@self.coefficients))
    
            # Stop SGD if tolerance is low
            if l_prev - l < self.tolerance:
                break
                
            self.log_losses.append(l)
            
        if self.fit_intercept == True:
            self.intercept = self.coefficients[0]
        else:
            self.intercept = 0.0 
            
        # Define residuals and deviance residuals
        self.residuals = y_hat - y
        self.deviance_residuals = np.sign(self.residuals) * np.sqrt(-2*(y*np.log(y_hat) + (1-y)*np.log(1-y_hat)))
        
        return self
        
       
    def predict(self, X, boundary=0.5):
        """
        Predict target using the fitted parameters of this Logistic Regression estimator.

        Parameters
        ----------
        X : array of shape M, k-1
            Values for regressors test data. (Don't add 1 for the intercept)

        Returns
        -------
        C : array of shape M
            Returns predicted values.
        """
        
        if hasattr(self, 'coefficients') == False:
                raise ValueError(
                     " This LogisticRegression instance is not fitted yet." \
                     " Call 'fit' method with appropriate X and y before using this predict function."
                 )
        
        if self.fit_intercept == True:
            X = np.append(np.ones((len(X),1)), X, axis=1)
        
        # Calculate predictions/y_hat.
        self.y_hat = self.sigmoid(X@self.coefficients)

        # If y_pred >= boundary: predict 1, else 0 and store predictions.
        self.y_pred = np.array([1 if i > boundary else 0 for i in self.y_hat])
        
        return self.y_pred
    
    def predict_prob(self, X, boundary=0.5):
        """
        Predict probability using the fitted parameters of this Logistic Regression estimator.

        Parameters
        ----------
        X : array of shape M, k-1
            Values for regressors test data. (Don't add 1 for the intercept)
            
        boundary : float
            Value of decision boundary. If estimated probability > boundary, predict 1.

        Returns
        -------
        C : array of shape M
            Returns predicted values.
        """
        
        if hasattr(self, 'coefficients') == False:
                raise ValueError(
                     " This LogisticRegression instance is not fitted yet." \
                     " Call 'fit' method with appropriate X and y before using this predict function."
                 )
        
        self.predict(X)
        
        probs = np.append(1-self.y_hat, self.y_hat, axis=1)
        
        return probs
    
    def accuracy(self, y, y_pred):
        """
        Print accuracy of fitted Logistic Regression estimator.

        Parameters
        ----------
        y : array of shape N
            Values for true test data.
        y_pred : array of shape N
            Values for predicted targets.

        Returns
        -------
        C : constant
            Returns accuracy.
        """
        
        if hasattr(self, 'coefficients') == False:
                raise ValueError(
                     " This LogisticRegression instance is not fitted yet." \
                     " Call 'fit' method with appropriate X and y before using this predict function."
                 )
                
        accuracy = sum(y==y_pred) / len(y)
        return accuracy

## Examples

In [2]:
import numpy as np
X = np.array([[-1, 1], [1, 2], [3, 4], [-3, 1], [-1, 0]])
y = np.array([1, 0, 4, 0, 0])

regfit = LogisticRegression(verbose=False, no_epochs=100000, tolerance=1e-10, fit_intercept=True).fit(X,y)
regfit.n_features

/var/folders/ng/83h0md1s73567d977zch9wnh0000gn/T/ipykernel_16601/944353457.py:221: RuntimeWarning: invalid value encountered in sqrt
  self.deviance_residuals = np.sign(self.residuals) * np.sqrt(-2*(y*np.log(y_hat) + (1-y)*np.log(1-y_hat)))


2

In [3]:
regfit.classes

array([0, 1, 4])

In [4]:
regfit.n_classes

3

In [5]:
regfit.coefficients

array([[1.04308925],
       [4.21019069],
       [5.25354458]])

In [6]:
regfit.intercept

array([1.04308925])

In [7]:
regfit.residuals

array([[-1.11257793e-01],
       [ 9.99999848e-01],
       [-3.00000000e+00],
       [ 1.81349561e-03],
       [ 4.08802670e-02]])

In [8]:
regfit.deviance_residuals

array([[-0.4856914 ],
       [ 5.60355676],
       [        nan],
       [ 0.06025184],
       [ 0.28892684]])

In [9]:
regfit.predict(X)

array([1, 1, 1, 0, 0])

In [10]:
regfit.predict_prob(X)

array([[0.73833645, 0.26166355],
       [0.50379972, 0.49620028],
       [0.12503625, 0.87496375],
       [0.75414522, 0.24585478],
       [0.87825167, 0.12174833]])

In [11]:
regfit.accuracy(y, regfit.y_pred)

0.8